<a href="https://colab.research.google.com/github/ucfilho/marquesgabi_2020/blob/master/Features_Determination_versao_nova_001_jan_29_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mahotas

In [2]:

!git clone https://github.com/ucfilho/marquesgabi #clonar do Github
%cd marquesgabi

fatal: destination path 'marquesgabi' already exists and is not an empty directory.
/content/marquesgabi


In [0]:
import numpy as np
import cv2
from skimage import data
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
from skimage import io, util
import mahotas.features.texture as mht
import mahotas.features
import os 
import zipfile
from skimage import filters
from skimage import exposure
import skimage.feature as sk
from google.colab import files
from numpy import linalg as LA
from scipy.signal import find_peaks
from scipy.signal import peak_prominences
from scipy.signal import peak_widths
import re
import Go2BlackWhite
import Go2Mahotas
%matplotlib inline

In [0]:
Size=5 # tamanho da foto
Sub_Size=int(Size/5) # tamanho do fracionamento
Row_Crop=1/2 # posicao do corte
Crop=int(Size*Row_Crop)

In [5]:
!wget http://ftp.feq.ufu.br/ubirajara/GrBuInd.zip
Transfere='GrBuInd.zip'

--2020-01-29 22:37:11--  http://ftp.feq.ufu.br/ubirajara/GrBuInd.zip
Resolving ftp.feq.ufu.br (ftp.feq.ufu.br)... 200.19.144.11
Connecting to ftp.feq.ufu.br (ftp.feq.ufu.br)|200.19.144.11|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 497627 (486K) [application/zip]
Saving to: ‘GrBuInd.zip.5’

GrBuInd.zip.5       100%[===================>] 485.96K   500KB/s    in 1.0s    

2020-01-29 22:37:12 (500 KB/s) - ‘GrBuInd.zip.5’ saved [497627/497627]



In [0]:
file_name = zipfile.ZipFile(Transfere, 'r')
file_name.extractall()

In [0]:
# print(img_name)

In [0]:
# ww contain all photos in black and white
ww,img_name=Go2BlackWhite.BlackWhite(Transfere,Size)
# plt.axis("off")
# plt.imshow(ww[0], cmap = "gray")

In [9]:
print(np.mean(ww[0]))

139.24


In [10]:
Num,Row,Col=np.asarray(ww).shape
Id=[]
Idx=[]

for i in range(Num):
  Media=np.mean(ww[0])
  k=np.where(ww[i].ravel() > Media)
  Idx.append(np.asarray(k).ravel())
  #Idx.append(Id[i])
print(Idx)


[array([ 2,  6,  7,  8,  9, 12, 13, 15, 16, 17, 18, 22, 24]), array([ 2,  3,  9, 15]), array([ 0,  1,  3,  4,  5,  6,  9, 11, 12, 14, 15, 16, 17, 20, 21, 24]), array([ 0,  1,  9, 14, 21]), array([0, 5]), array([15]), array([ 0,  1,  2,  4,  5,  6,  7,  8,  9, 10, 11, 12, 14, 15, 16, 17, 19,
       20, 21, 22, 23, 24]), array([ 0, 20, 21, 22]), array([ 0,  1,  2,  3,  5,  6, 14]), array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24]), array([ 2,  3,  9, 15, 20, 22, 23]), array([ 1,  2,  4,  6,  7,  8,  9, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21,
       23]), array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 21, 24]), array([ 0,  2,  3,  4,  6,  7,  8,  9, 11, 12, 13, 14, 15, 16, 17, 18, 20,
       21, 22, 23, 24]), array([ 1,  5,  6,  9, 11, 14, 16, 18, 21, 22, 23]), array([ 0,  1,  2,  4,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 18, 20,
       22, 24]), array([ 0,  1,  2,  3,  4,  5, 

In [19]:
Quant=Size*Size
for i in range(Num):
  print(len(np.asarray(Idx[i])))
  print(np.asarray(Idx[i]))
  print(len(np.asarray(Idx[i]))/Quant)
  #Idx.append(Id[i])

13
[ 2  6  7  8  9 12 13 15 16 17 18 22 24]
0.52
4
[ 2  3  9 15]
0.16
16
[ 0  1  3  4  5  6  9 11 12 14 15 16 17 20 21 24]
0.64
5
[ 0  1  9 14 21]
0.2
2
[0 5]
0.08
1
[15]
0.04
22
[ 0  1  2  4  5  6  7  8  9 10 11 12 14 15 16 17 19 20 21 22 23 24]
0.88
4
[ 0 20 21 22]
0.16
7
[ 0  1  2  3  5  6 14]
0.28
25
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24]
1.0
7
[ 2  3  9 15 20 22 23]
0.28
18
[ 1  2  4  6  7  8  9 11 12 13 14 15 16 17 18 20 21 23]
0.72
21
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 21 24]
0.84
21
[ 0  2  3  4  6  7  8  9 11 12 13 14 15 16 17 18 20 21 22 23 24]
0.84
11
[ 1  5  6  9 11 14 16 18 21 22 23]
0.44
19
[ 0  1  2  4  6  7  8  9 10 11 12 13 14 15 16 18 20 22 24]
0.76
18
[ 0  1  2  3  4  5  6  8  9 11 12 13 16 17 18 19 20 21]
0.72
5
[ 0  6 11 16 23]
0.2
3
[1 3 8]
0.12
21
[ 1  2  3  4  5  6  7  8 11 12 13 14 16 17 18 19 20 21 22 23 24]
0.84
21
[ 0  1  2  3  4  6  7  8  9 11 12 13 14 16 17 18 19 21 22 23 24]
0.84
11
[ 0  1  3  

In [18]:
for i in Idx:
  print(i)
  print(len(i))
  

[ 2  6  7  8  9 12 13 15 16 17 18 22 24]
13
[ 2  3  9 15]
4
[ 0  1  3  4  5  6  9 11 12 14 15 16 17 20 21 24]
16
[ 0  1  9 14 21]
5
[0 5]
2
[15]
1
[ 0  1  2  4  5  6  7  8  9 10 11 12 14 15 16 17 19 20 21 22 23 24]
22
[ 0 20 21 22]
4
[ 0  1  2  3  5  6 14]
7
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24]
25
[ 2  3  9 15 20 22 23]
7
[ 1  2  4  6  7  8  9 11 12 13 14 15 16 17 18 20 21 23]
18
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 21 24]
21
[ 0  2  3  4  6  7  8  9 11 12 13 14 15 16 17 18 20 21 22 23 24]
21
[ 1  5  6  9 11 14 16 18 21 22 23]
11
[ 0  1  2  4  6  7  8  9 10 11 12 13 14 15 16 18 20 22 24]
19
[ 0  1  2  3  4  5  6  8  9 11 12 13 16 17 18 19 20 21]
18
[ 0  6 11 16 23]
5
[1 3 8]
3
[ 1  2  3  4  5  6  7  8 11 12 13 14 16 17 18 19 20 21 22 23 24]
21
[ 0  1  2  3  4  6  7  8  9 11 12 13 14 16 17 18 19 21 22 23 24]
21
[ 0  1  3  4  9 14 18 19 22 23 24]
11
[ 0  1  2  3  4  5  6  7  8  9 10 12 13 14 15 16 18 20 21 22]
20
[ 1  3  4  5 

TypeError: ignored

In [0]:
# Find the list elements starting with specific letter
subs = 'B'
Buraco = [x for x in img_name if re.search(subs, x)] 
subs = 'G'
Grao = [x for x in img_name if re.search(subs, x)] 
subs = 'I'
Indef = [x for x in img_name if re.search(subs, x)] 

In [0]:
# find df texture for a given picture
df=Go2Mahotas.Mahotas(ww[0],Size,Sub_Size,Crop,img_name)
print(df)

In [0]:
j=0
cont=-1
Posicao_Buraco=[]
for i in img_name:
  cont=cont+1
  for k in Buraco:
    if(i==k):
      Posicao_Buraco.append(cont)
j=0
cont=-1
Posicao_Grao=[]
for i in img_name:
  cont=cont+1
  for k in Grao:
    if(i==k):
      Posicao_Grao.append(cont)

j=0
cont=-1
Posicao_Indef=[]
for i in img_name:
  cont=cont+1
  for k in Indef:
    if(i==k):
      Posicao_Indef.append(cont)

# print(Posicao_Grao)
# print(Posicao_Buraco)
# print(Posicao_Indef)


In [0]:
# compara constrast dos tres primeiros buracos
Num=len(Posicao_Buraco)
Matriz_X_Buraco=[]
#Num=3
for i in range(Num):
  k=Posicao_Buraco[i]
  df=Go2Mahotas.Mahotas(ww[k],Size,Sub_Size,Crop,img_name)
  Matriz_X_Buraco.append(np.array(df['constrast']).ravel())
  #plt.figure(i+1)
  #plt.plot(df['constrast'])
  #print(Matriz_X_Buraco)
Contraste_Buraco=pd.DataFrame(Matriz_X_Buraco)

In [0]:
# compara constrast dos graos
Num=len(Posicao_Grao)
Matriz_X_Grao=[]
for i in range(Num):
  k=Posicao_Grao[i]
  df=Go2Mahotas.Mahotas(ww[k],Size,Sub_Size,Crop,img_name)
  Matriz_X_Grao.append(np.array(df['constrast']).ravel())
  #plt.figure(i+1)
  #plt.plot(df['constrast'])
  #print(Matriz_X_Grao)
Contraste_Grao=pd.DataFrame(Matriz_X_Grao)

In [0]:
# compara constrast dos indefinidos
Num=len(Posicao_Indef)
Matriz_X_Indef=[]
for i in range(Num):
  k=Posicao_Indef[i]
  df=Go2Mahotas.Mahotas(ww[k],Size,Sub_Size,Crop,img_name)
  Matriz_X_Indef.append(np.array(df['constrast']).ravel())
  #plt.figure(i+1)
  #plt.plot(df['constrast'])
  #print(Matriz_X_Indef)
Contraste_Indef=pd.DataFrame(Matriz_X_Indef)

In [0]:
print(Contraste_Buraco.head())

In [0]:
print(Contraste_Grao.head())

In [0]:
print(Contraste_Indef.head())

In [0]:
# found drive
from google.colab import drive
drive.mount('drive')

In [0]:
# get csv file
Contraste_Buraco.to_csv('Contraste_Buraco_01_jan_2020.csv')

In [0]:
# get csv file
Contraste_Grao.to_csv('Contraste_Grao_01_jan_2020.csv')

In [0]:
# get csv file
Contraste_Indef.to_csv('Contraste_Indef_01_jan_2020.csv')

In [0]:
# save in drive
!cp Contraste_Buraco_01_jan_2020.csv drive/My\ Drive/Maria_Gabriela_Textura_dados_jan_2020/

In [0]:
# save in drive
!cp Contraste_Grao_01_jan_2020.csv drive/My\ Drive/Maria_Gabriela_Textura_dados_jan_2020/

In [0]:
# save in drive
!cp Contraste_Indef_01_jan_2020.csv drive/My\ Drive/Maria_Gabriela_Textura_dados_jan_2020/